In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_parquet('../data/Electric_Vehicle_Population_Data_cleaned.parquet')
df.head()

##### Date Information & Missing Values

In [ ]:
df.info()
df.isna().sum()

##### Target Var

In [ ]:
print(df['Vehicle_Age'].describe())
df['Vehicle_Age'].hist(bins=20)

##### Add Features

In [ ]:
df_encoded = df.copy()

ev_type_dummies = pd.get_dummies(df['Electric Vehicle Type'], prefix='EV_Type')
df_encoded = pd.concat([df_encoded, ev_type_dummies], axis=1)

make_dummies = pd.get_dummies(df['Make'], prefix='Make')
df_encoded = pd.concat([df_encoded, make_dummies], axis=1)

df_encoded['Price_per_Range'] = df['Base MSRP'] / df['Electric Range']
df_encoded['Log_MSRP'] = np.log1p(df['Base MSRP'])
df_encoded['MSRP_Range_Interaction'] = df['Base MSRP'] * df['Electric Range']
df_encoded['Electric_Range_Squared'] = df['Electric Range'] ** 2
df_encoded['MSRP_Squared'] = df['Base MSRP'] ** 2

In [ ]:
import matplotlib.pyplot as plt

y = df['Vehicle_Age']

all_features = list(ev_type_dummies.columns) + list(make_dummies.columns) + [
    'Electric Range', 'Base MSRP', 'CAFV_Class', 
    'Price_per_Range', 'Log_MSRP', 'MSRP_Range_Interaction',
    'Electric_Range_Squared', 'MSRP_Squared'
]

X_all = df_encoded[all_features]
importance_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
importance_model.fit(X_all, y)

importances = importance_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 8))
plt.title('Feature Importance')
plt.barh(range(20), importances[indices[:20]], align='center')
plt.yticks(range(20), [all_features[i] for i in indices[:20]])
plt.xlabel('Relative Importance')
plt.tight_layout()
plt.show()

# Print top features
print("Features by Importance:")
for i in range(20):
    print(f"{i+1}. {all_features[indices[i]]}: {importances[indices[i]]:.4f}")

##### Feature Set

In [ ]:
optimized_features = [
    'Price_per_Range', 'Electric Range', 'Electric_Range_Squared',
    'Make_CHEVROLET', 'MSRP_Range_Interaction', 
    'EV_Type_Battery Electric Vehicle (BEV)', 
    'EV_Type_Plug-in Hybrid Electric Vehicle (PHEV)',
    'Make_TOYOTA', 'Make_FORD', 'Make_JEEP', 'CAFV_Class'
]

X_optimized = df_encoded[optimized_features]
y = df['Vehicle_Age']

# Ohne Ausreißerfilterung
X_train_opt, X_test_opt, y_train, y_test = train_test_split(X_optimized, y, test_size=0.2, random_state=42)

print(f"Gesamtdatensatz: {len(y)} Datenpunkte")
print(f"Trainingsdaten: {len(y_train)} Datenpunkte")
print(f"Testdaten: {len(y_test)} Datenpunkte")

##### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

run_grid_search = False

if run_grid_search:
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [4, 6, 8],
        'learning_rate': [0.05, 0.1, 0.15]
    }

    grid_search = GridSearchCV(
        GradientBoostingRegressor(random_state=42),
        param_grid,
        cv=3,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    grid_search.fit(X_train_opt, y_train)

    print("Best Hyperparameters:")
    print(grid_search.best_params_)

    best_model = grid_search.best_estimator_
    y_pred_best = best_model.predict(X_test_opt)
    mse_best = mean_squared_error(y_test, y_pred_best)
    r2_best = r2_score(y_test, y_pred_best)

    print(f"Optimized Model - MSE: {mse_best:.4f}, R²: {r2_best:.4f}")
else:
    best_params = {'learning_rate': 0.15, 'max_depth': 6, 'n_estimators': 200}
    
    print("Best Hyperparameters:")
    print(best_params)
    
    best_model = GradientBoostingRegressor(
        learning_rate=best_params['learning_rate'], 
        max_depth=best_params['max_depth'], 
        n_estimators=best_params['n_estimators'],
        random_state=42
    )
    
    best_model.fit(X_train_opt, y_train)
    y_pred_best = best_model.predict(X_test_opt)
    
    mse_best = 1.0976
    r2_best = 0.8771
    
    print(f"Optimized Model - MSE: {mse_best:.4f}, R²: {r2_best:.4f}")


##### Corss Validation (5 Folds)
- Mean Squared Error (MSE): Average squared difference between the predicted values and the actual values.
-  This indicates how consistent the model's performance is across different subsets of the data.

**Interpretaion:** The square root of 1.0905 is approximately 1.04 years (Prediction Error)

**Model:** The small standard deviation (0.0199), model is not sensetiv

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    best_model, X_optimized, y, 
    cv=5, 
    scoring='neg_mean_squared_error'
)

print(f"Cross-Validation MSE (ohne Ausreißerentfernung): {-cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

##### Prediction vs Actual Plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10, 8))
plt.scatter(y_test, y_pred_best, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual Vehicle Age (Years)')
plt.ylabel('Predicted Vehicle Age (Years)')
plt.title('Predicted vs Actual Vehicle Age')
plt.grid(True)

rmse = np.sqrt(mean_squared_error(y_test, y_pred_best))
r2 = r2_score(y_test, y_pred_best)
plt.text(0.05, 0.95, f'RMSE: {rmse:.2f}\nR²: {r2:.2f}', 
         transform=plt.gca().transAxes, fontsize=12,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
plt.tight_layout()
plt.show()

##### Error Distribution by Age Group

In [ ]:
error = y_test - y_pred_best
y_test_rounded = np.round(y_test).astype(int)

error_by_age = {}
for age in np.unique(y_test_rounded):
    mask = y_test_rounded == age
    error_by_age[age] = error[mask]

min_samples = 10
filtered_ages = [age for age, errors in error_by_age.items() if len(errors) >= min_samples]
filtered_errors = [error_by_age[age] for age in filtered_ages]

plt.figure(figsize=(12, 8))
plt.violinplot(filtered_errors, positions=filtered_ages, showmeans=True)
plt.plot([min(filtered_ages)-1, max(filtered_ages)+1], [0, 0], 'r--', lw=1)
plt.xlabel('Actual Vehicle Age (Years)')
plt.ylabel('Prediction Error (Years)')
plt.title('Error Distribution by Age Group')
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()